# EMNIST

In [13]:
%load_ext autoreload
%autoreload 2
# Import dependencies
import os
import importlib
import numpy as np
import matplotlib.pyplot as plt
import time
import torchvision
import seaborn as sns
import numpy as np
import pytorch_lightning as pl
import torch.nn.functional as F
import torchvision
from torchvision import datasets, transforms
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision.utils import save_image
import torch
from tqdm import tqdm
from datetime import datetime
import yaml
import h5py
from copy import deepcopy
import ambiguous.models.cvae
from ambiguous.models.cvae import *
from ambiguous.dataset.dataset import partition_dataset
import wandb
from pytorch_lightning.loggers import WandbLogger

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
TRAIN_CVAE = True
LOCAL_CKPT = False

# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Seed for reproducibility
torch.manual_seed(42)
print(device)
root='/home/mila/n/nizar.islah/expectation-clamp/'

cuda


In [4]:
transform = transforms.Compose([
    transforms.ToTensor(),
    lambda x: x.rot90(1,[2,1]).flip(2)
])
dataset = datasets.EMNIST(root=root, download=False, split='byclass', train=True, transform=transform)
new_dataset = partition_dataset(dataset, range(10, 36))
train_set, val_set = torch.utils.data.random_split(new_dataset, [round(0.8*len(new_dataset)), round(0.2*len(new_dataset))])
test_set = datasets.EMNIST(root=root, download=False, split='byclass', train=False, transform=transform)
# Dataloaders
batch_size = 64
train_loader = DataLoader(train_set, batch_size=batch_size, num_workers=2, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size, num_workers=2)
test_loader = DataLoader(test_set, batch_size=batch_size, num_workers=2)

In [5]:
s = range(100)
t = tqdm(total=len(s))
for x in s:
    t.update()
t.refresh()  # force print final state

t.reset()  # reuse bar
for x in s:
    t.update()
t.close()  # close the bar permanently

100%|██████████| 100/100 [00:00<00:00, 50225.17it/s] 


In [6]:
x,t=next(iter(train_loader))
torchvision.utils.save_image(x, 'image.pdf', nrow=8)

In [7]:
n_cls=26
img_size=28
lr=1e-3
num_epochs=50
latent_dim = 20

onehot = torch.zeros(n_cls, n_cls).to(device)
onehot = onehot.scatter_(1, torch.LongTensor(range(n_cls)).view(n_cls,1).to(device), 1).view(n_cls, n_cls, 1, 1)
fill = torch.zeros([n_cls, n_cls, img_size, img_size]).to(device)
for i in range(n_cls):
    fill[i, i, :, :] = 1

if TRAIN_CVAE:
    wandb.init(project="EMNIST_CVAE", entity="team-nizar")
    model = Conv_CVAE(
    latent_dim = latent_dim,
        n_cls=n_cls
    ).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss(reduction='sum')
    n_train = len(train_loader)
    n_val = len(val_loader)
    for i in tqdm(range(num_epochs)):
        running_loss = 0
        for idx, (images, labels) in tqdm(enumerate(train_loader)):
            images = images.to(device)
            labels = labels.to(device)
            labels_fill_ = fill[labels]
            y_ = (torch.rand(images.size(0), 1) * n_cls).type(torch.LongTensor).squeeze().to(device)
            y_label_ = onehot[y_]
            rec, mu, logvar = model((images, labels_fill_, y_label_))
            loss_dict = model.loss_function(rec, images, mu, logvar)
            running_loss += loss_dict['loss'].item()/batch_size
            optimizer.zero_grad()
            loss_dict['loss'].backward()
            optimizer.step()
            wandb.log(loss_dict)
        val_loss = 0
        for _, (images, labels) in tqdm(enumerate(val_loader)):
            with torch.no_grad():
                images = images.to(device)
                labels = labels.to(device)
                labels_fill_ = fill[labels]
                y_ = (torch.rand(images.size(0), 1) * n_cls).type(torch.LongTensor).squeeze()
                y_label_ = onehot[y_]
                rec, mu, logvar = model((images, labels_fill_, y_label_))
                torchvision.utils.save_image(rec, "reconstruction_valid.pdf")
                loss_dict = model.loss_function(rec, images, mu, logvar)
                val_loss += loss_dict['loss'].item()/batch_size
                loss_dict = {'loss_val':loss_dict['loss'], 'kld_val':loss_dict['kld'], 'rec_val':loss_dict['rec']}
                wandb.log(loss_dict)
        torch.save(model, 'ConvCVAE.pth')
        print(f"Epoch: {i+1} \t Train Loss: {running_loss:.2f} \t Val Loss: {val_loss:.2f}")
    
else:
    if LOCAL_CKPT:
        ckpt_path = ""
        ckpt = torch.load(ckpt_path)
    else:
        run = wandb.init()
        artifact = run.use_artifact('team-nizar/EMNIST_CVAE/model-pkia9rhl:v29', type='model')
        artifact_dir = artifact.download()
        ckpt = torch.load(artifact_dir+'/model.ckpt')
        model = EMNIST_CVAE(latent_dim, enc_layers, dec_layers, n_classes=26, conditional=True).to(device)
        model.load_state_dict(ckpt['state_dict'])
    
model.eval()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: team-nizar (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.17 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


26 16


  0%|          | 0/50 [00:00<?, ?it/s]
0it [00:00, ?it/s]
1it [00:00,  7.61it/s]
6it [00:00, 28.10it/s]
12it [00:00, 39.62it/s]
18it [00:00, 45.02it/s]
24it [00:00, 48.58it/s]
30it [00:00, 50.17it/s]
36it [00:00, 52.51it/s]
42it [00:00, 54.12it/s]
48it [00:01, 54.73it/s]
54it [00:01, 55.43it/s]
60it [00:01, 56.03it/s]
66it [00:01, 56.43it/s]
72it [00:01, 56.52it/s]
78it [00:01, 55.43it/s]
84it [00:01, 55.47it/s]
90it [00:01, 55.02it/s]
96it [00:01, 55.52it/s]
102it [00:01, 56.03it/s]
108it [00:02, 55.77it/s]
114it [00:02, 55.95it/s]
120it [00:02, 51.03it/s]
126it [00:02, 50.69it/s]
132it [00:02, 52.76it/s]
138it [00:02, 51.99it/s]
144it [00:02, 52.89it/s]
150it [00:02, 54.13it/s]
156it [00:02, 55.15it/s]
162it [00:03, 55.91it/s]
168it [00:03, 55.82it/s]
174it [00:03, 55.22it/s]
180it [00:03, 55.87it/s]
186it [00:03, 56.27it/s]
192it [00:03, 56.63it/s]
198it [00:03, 56.74it/s]
204it [00:03, 55.81it/s]
210it [00:03, 56.77it/s]
216it [00:04, 57.42it/s]
222it [00:04, 56.20it/s]
228it [00:0

Epoch: 1 	 Train Loss: 97300.41 	 Val Loss: 21499.44



0it [00:00, ?it/s]
4it [00:00, 34.50it/s]
10it [00:00, 45.08it/s]
16it [00:00, 48.12it/s]
22it [00:00, 51.00it/s]
28it [00:00, 53.16it/s]
34it [00:00, 54.14it/s]
40it [00:00, 54.06it/s]
46it [00:00, 55.18it/s]
52it [00:00, 56.14it/s]
58it [00:01, 56.73it/s]
64it [00:01, 57.12it/s]
70it [00:01, 57.08it/s]
76it [00:01, 57.29it/s]
82it [00:01, 57.57it/s]
88it [00:01, 57.68it/s]
94it [00:01, 57.12it/s]
100it [00:01, 56.44it/s]
106it [00:01, 56.66it/s]
112it [00:02, 56.65it/s]
118it [00:02, 52.26it/s]
124it [00:02, 52.45it/s]
130it [00:02, 53.26it/s]
136it [00:02, 53.04it/s]
142it [00:02, 53.24it/s]
148it [00:02, 54.46it/s]
154it [00:02, 55.37it/s]
160it [00:02, 55.91it/s]
166it [00:03, 55.17it/s]
172it [00:03, 55.76it/s]
178it [00:03, 56.16it/s]
184it [00:03, 56.18it/s]
190it [00:03, 54.67it/s]
196it [00:03, 54.00it/s]
202it [00:03, 53.89it/s]
208it [00:03, 55.01it/s]
214it [00:03, 54.76it/s]
220it [00:04, 54.61it/s]
226it [00:04, 54.78it/s]
232it [00:04, 55.14it/s]
238it [00:04, 52.71it/

...

Epoch: 29 	 Train Loss: 70238.38 	 Val Loss: 17706.38



0it [00:00, ?it/s]
2it [00:00, 15.36it/s]
8it [00:00, 37.58it/s]
14it [00:00, 45.75it/s]
20it [00:00, 49.96it/s]
26it [00:00, 52.65it/s]
32it [00:00, 54.35it/s]
38it [00:00, 55.80it/s]
44it [00:00, 56.91it/s]
50it [00:00, 57.10it/s]
56it [00:01, 57.26it/s]
62it [00:01, 57.36it/s]
68it [00:01, 55.14it/s]
74it [00:01, 53.34it/s]
80it [00:01, 53.51it/s]
86it [00:01, 53.01it/s]
92it [00:01, 53.93it/s]
98it [00:01, 54.02it/s]
104it [00:01, 54.76it/s]
110it [00:02, 55.65it/s]
116it [00:02, 56.22it/s]
122it [00:02, 54.80it/s]
128it [00:02, 54.37it/s]
134it [00:02, 55.40it/s]
140it [00:02, 56.13it/s]
146it [00:02, 56.73it/s]
152it [00:02, 56.63it/s]
158it [00:02, 57.07it/s]
164it [00:03, 56.72it/s]
170it [00:03, 56.99it/s]
176it [00:03, 55.08it/s]
182it [00:03, 54.83it/s]
188it [00:03, 51.93it/s]
194it [00:03, 49.37it/s]
200it [00:03, 50.73it/s]
206it [00:03, 51.81it/s]
212it [00:03, 52.40it/s]
218it [00:04, 52.77it/s]
224it [00:04, 53.67it/s]
230it [00:04, 53.95it/s]
236it [00:04, 54.60it/s]

Epoch: 30 	 Train Loss: 70166.78 	 Val Loss: 17694.56



0it [00:00, ?it/s]
1it [00:00,  3.85it/s]
2it [00:00,  2.74it/s]
7it [00:00, 11.56it/s]
13it [00:00, 21.33it/s]
18it [00:01, 27.46it/s]
24it [00:01, 34.51it/s]
30it [00:01, 40.53it/s]
36it [00:01, 44.89it/s]
42it [00:01, 48.30it/s]
48it [00:01, 50.47it/s]
54it [00:01, 52.54it/s]
60it [00:01, 53.94it/s]
66it [00:01, 54.94it/s]
72it [00:01, 55.83it/s]
78it [00:02, 55.73it/s]
84it [00:02, 55.63it/s]
90it [00:02, 56.23it/s]
96it [00:02, 56.30it/s]
102it [00:02, 56.15it/s]
108it [00:02, 56.71it/s]
114it [00:02, 57.27it/s]
120it [00:02, 53.71it/s]
126it [00:02, 51.73it/s]
132it [00:03, 52.80it/s]
138it [00:03, 53.10it/s]
144it [00:03, 54.40it/s]
150it [00:03, 54.90it/s]
156it [00:03, 54.67it/s]
162it [00:03, 55.20it/s]
168it [00:03, 54.58it/s]
174it [00:03, 55.11it/s]
180it [00:03, 55.26it/s]
186it [00:04, 55.88it/s]
192it [00:04, 56.52it/s]
198it [00:04, 56.91it/s]
204it [00:04, 57.23it/s]
210it [00:04, 57.49it/s]
216it [00:04, 56.24it/s]
222it [00:04, 55.73it/s]
228it [00:04, 55.20it/s]
2

KeyboardInterrupt: 

In [7]:
onehot

tensor([[[[1.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]]],


        [[[0.]],

         [[1.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]]],


        [[[0.]],

         [[0.]],

         [[1.]],

      

In [8]:
model.eval()
for _, (images, labels) in enumerate(val_loader):
    with torch.no_grad():
        images = images.to(device)
        labels = labels.to(device)
        labels_fill_ = fill[labels]
        y_ = (torch.rand(images.size(0), 1) * n_cls).type(torch.LongTensor).squeeze()
        y_label_ = onehot[y_]
        rec, mu, logvar = model((images, labels_fill_, y_label_))
    break

torchvision.utils.save_image(rec, "example.pdf")

In [14]:
torch.cuda.empty_cache()

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [11]:
y_label_

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [10]:
for _, (images, labels) in enumerate(val_loader):
    with torch.no_grad():
        images = images.to(device)
        labels = labels.to(device)
        labels_fill_ = fill[labels]
        y_ = (torch.rand(images.size(0), 2) * n_cls).type(torch.LongTensor).squeeze()
        y_label_ = onehot[y_]
        rec, mu, logvar = model((images, labels_fill_, y_label_))
    break
    
torchvision.utils.save_image(rec, "example_ambi.pdf")

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.